In [1]:
DATA_NAME = 'segmentAnything-full' 
TRANSFORM = 'wavelet-vertical'
CHANNEL = 'blue'

In [2]:
path_list = DATA_NAME.split("-") + TRANSFORM.split("-")
if CHANNEL:
    path_list.append(CHANNEL)
print(f"Name the notebook:\nindependence_{'_'.join(path_list[::-1])}.ipynb")
FULL_DATA_NAME='-'.join(path_list)


import git
from pathlib import Path
import os
CWD = os.getcwd()
ROOT_DIR = Path(git.Repo('.', search_parent_directories=True).working_tree_dir)

Path(os.path.join(CWD, "CSVs")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "plots")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "cache")).mkdir(exist_ok=True)

GROUP = 'layer' if TRANSFORM.split("-")[0] == 'wavelet' else ('band' if TRANSFORM.split("-")[0] == 'fourier' else 'error')
RERUN = False
SKIP_OPTIMIZE_STEP = False

Name the notebook:
independence_blue_vertical_wavelet_full_segmentAnything.ipynb


In [3]:
os.chdir(os.path.join(ROOT_DIR, "utilities"))
from testing import * # If MATLAB is not installed, open utilities and set to False
from plotting import *
os.chdir(CWD)
np.random.seed(0)

In [4]:
from scipy import spatial
from sklearn.decomposition import PCA

In [5]:
group_data_map = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}.pickle'))
group_total_samples = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}-size.pickle'))

In [6]:
if 'fourier' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)#[::3]
elif 'wavelet' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)
elif 'learned' in TRANSFORM:
    nonskewed_df = pd.read_csv(os.path.join(ROOT_DIR, 'learned-filters', 'nonskewed_filter_idxs_df.csv')).set_index(['dataset', 'num_images', 'num_bootstrap'])
    nonskewed_filter_idxs = eval(nonskewed_df.loc[DATA_NAME].sort_values('num_images', ascending=False)['nonskewed_filter_idxs'].iloc[0]) 
    GROUPS = nonskewed_filter_idxs # can set to filter_group_map.keys() to include all prepared filters

In [7]:
n_bootstrap = int(1e5) 
bootstrap_size = int(1e4)


In [8]:

cov_matrix = np.zeros((len(GROUPS), len(GROUPS)))
for _ in tqdm(range(n_bootstrap)):
    X = np.zeros((bootstrap_size, len(GROUPS)))
    for i in range(len(GROUPS)):
        group = GROUPS[i]
        X[:, i] = np.random.choice(group_data_map[group], size=(bootstrap_size), replace=True)
        cov_matrix += np.cov(X, rowvar=False)
cov_matrix /= n_bootstrap

  0%|          | 0/100000 [00:00<?, ?it/s]

In [9]:



# Convert covariance matrix to DataFrame for better formatting
cov_df = pd.DataFrame(cov_matrix, index=GROUPS, columns=GROUPS)
cov_df.to_csv(os.path.join(CWD, "CSVs", f'covariance_matrix.csv'))
cov_df.round(2)

,2,3,4,5,6,7,8,9,10
2,80531.89,-0.46,0.01,0.30,0.02,-0.04,0.00,-0.00,-0.00
3,-0.46,19416.97,-0.13,0.05,-0.05,0.01,0.02,0.00,0.00
4,0.01,-0.13,4636.49,0.08,0.00,0.00,-0.01,-0.00,0.00
5,0.30,0.05,0.08,1018.14,-0.00,-0.00,0.00,0.00,0.00
6,0.02,-0.05,0.00,-0.00,203.79,-0.00,-0.00,0.00,0.00
7,-0.04,0.01,0.00,-0.00,-0.00,37.11,0.00,0.00,0.00
8,0.00,0.02,-0.01,0.00,-0.00,0.00,6.17,0.00,0.00
9,-0.00,0.00,-0.00,0.00,0.00,0.00,0.00,0.95,-0.00
10,-0.00,0.00,0.00,0.00,0.00,0.00,0.00,-0.00,0.11


In [10]:
corr_matrix = cov_matrix / np.sqrt(np.outer(np.diag(cov_matrix), np.diag(cov_matrix)))
corr_df = pd.DataFrame(corr_matrix, index=GROUPS, columns=GROUPS)
corr_df.round(5)

,2,3,4,5,6,7,8,9,10
2,1.00000,-0.00001,0.00000,0.00003,0.00000,-0.00002,0.00001,-0.00001,-0.00000
3,-0.00001,1.00000,-0.00001,0.00001,-0.00003,0.00001,0.00005,0.00001,0.00000
4,0.00000,-0.00001,1.00000,0.00004,0.00000,0.00000,-0.00003,-0.00000,0.00001
5,0.00003,0.00001,0.00004,1.00000,-0.00000,-0.00002,0.00005,0.00000,0.00001
6,0.00000,-0.00003,0.00000,-0.00000,1.00000,-0.00002,-0.00000,0.00002,0.00000
7,-0.00002,0.00001,0.00000,-0.00002,-0.00002,1.00000,0.00001,0.00000,0.00000
8,0.00001,0.00005,-0.00003,0.00005,-0.00000,0.00001,1.00000,0.00003,0.00000
9,-0.00001,0.00001,-0.00000,0.00000,0.00002,0.00000,0.00003,1.00000,-0.00000
10,-0.00000,0.00000,0.00001,0.00001,0.00000,0.00000,0.00000,-0.00000,1.00000


In [11]:
np.linalg.norm(cov_matrix - np.diag(np.diag(cov_matrix)))  # Remove diagonal elements for visualization

0.8152802204881585

In [12]:
pca = PCA()
pca.fit(cov_matrix)

print("Singular values (explained variance):")
print(pca.explained_variance_)

print("\nPrincipal components (eigenvectors):")
eigenvectors = pca.components_
eigenvectors_df = pd.DataFrame(eigenvectors)
eigenvectors_df

Singular values (explained variance):
[7.21331912e+08 4.12506416e+07 2.30365434e+06 1.08008996e+05
 4.15504327e+03 1.29214349e+02 3.18483613e+00 5.71127956e-02
 2.27013222e-35]

Principal components (eigenvectors):


,0,1,2,3,4,5,6,7,8
0,0.999463,-0.031941,-0.007153,-0.001562,-0.000313,-0.000058,-0.000009,-0.000001,-1.761085e-07
1,0.031647,0.998809,-0.036355,-0.007482,-0.001496,-0.000271,-0.000044,-0.000007,-8.258545e-07
2,0.008228,0.035784,0.998550,-0.038652,-0.007322,-0.001330,-0.000223,-0.000034,-4.044203e-06
3,0.002097,0.008730,0.037996,0.998331,-0.041917,-0.007281,-0.001205,-0.000186,-2.222551e-05
4,0.000504,0.002094,0.008769,0.041194,0.997962,-0.047285,-0.007548,-0.001161,-1.389556e-04
5,0.000115,0.000475,0.001989,0.009071,0.046414,0.997205,-0.057170,-0.008500,-1.015504e-03
6,0.000025,0.000104,0.000440,0.001996,0.009996,0.055890,0.995224,-0.078852,-9.100957e-03
7,0.000006,0.000024,0.000101,0.000459,0.002291,0.012595,0.076592,0.989753,-1.198284e-01
8,0.000001,0.000006,0.000024,0.000111,0.000554,0.003046,0.018309,0.118735,9.927524e-01


In [13]:
cos_dist = np.diag(spatial.distance.cdist(eigenvectors, np.eye(len(GROUPS)), metric='cosine'))
cos_dist = cos_dist = [np.min(row) for row in cos_dist]
cos_dist


[0.0005371053866383146,
 0.0011914777736893312,
 0.0014498640197793122,
 0.0016693209083383787,
 0.0020384304244479567,
 0.00279508931601391,
 0.004775591879186636,
 0.010247135343396274,
 0.007247628593837097]